In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM, PeftModel
import torch

In [5]:
tokenizer = AutoTokenizer.from_pretrained('unsloth/Llama-3.2-1B-Instruct')

In [6]:
tokenizer.push_to_hub('mesolitica/malaysian-Llama-3.2-1B-Instruct')

CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-Llama-3.2-1B-Instruct/commit/f5716b03084ce750352fbdd5568ad4c7f2c6e4e8', commit_message='Upload tokenizer', commit_description='', oid='f5716b03084ce750352fbdd5568ad4c7f2c6e4e8', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
streamer = TextStreamer(tokenizer)

In [6]:
ori_model = AutoModelForCausalLM.from_pretrained(
    'unsloth/Llama-3.2-1B-Instruct', tie_word_embeddings=False
).cuda()

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at unsloth/Llama-3.2-1B-Instruct and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
ori_model.lm_head.weight.data = ori_model.model.embed_tokens.weight.data.clone()

In [8]:
state_dict = ori_model.state_dict()

In [9]:
from safetensors import safe_open

f = safe_open(f"1b/adapter_model.safetensors", framework="pt", device=0)

In [10]:
keys = f.keys()
keys = sorted(list(set([k.split('.lora')[0] for k in keys if '.lora' in k])))

In [11]:
from tqdm import tqdm

for k in tqdm(keys):
    k_ori = k.replace('base_model.model.', '') + '.weight'
    if 'embed_tokens' in k:
        post_A = '.lora_embedding_A'
        post_B = '.lora_embedding_B'
    else:
        post_A = '.lora_A.weight'
        post_B = '.lora_B.weight'
    A = k + post_A
    B = k + post_B
    
    W = state_dict[k_ori]
    if 'embed_tokens' not in k:
        W = W.t()
        
    A = f.get_tensor(A)
    B = f.get_tensor(B)
    with torch.no_grad():
        W.addmm_(A.t(), B.t(), alpha = 2)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 114/114 [00:00<00:00, 309.27it/s]


In [12]:
# model = AutoPeftModelForCausalLM.from_pretrained(
#     'mesolitica/lora-embedding-256-llama3.2-3b-multipack', torch_dtype=torch.bfloat16,
# )
# _ = model.cuda()

In [14]:
d = [
    {'role': 'user', 'content': 'camne nk selesaikan masalah hutang negara'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=512,
    top_p=0.95,
    top_k=50,
    temperature=0.0,
    do_sample=False,
    repetition_penalty=1.05,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 21 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk selesaikan masalah hutang negara<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Terima kasih atas pertanyaan anda mengenai cara untuk menyelesaikan masalah hutang negara. Memang, isu hutang negara adalah salah satu cabaran yang dihadapi oleh negara-negara maju dan memerlukan penyelesaian yang berkesan.

Untuk menyelesaikan masalah hutang negara, terdapat beberapa langkah yang boleh diambil:

1. Meningkatkan pengurusan kewangan: Negara-negara maju perlu meningkatkan pengurusan kewangan mereka dengan menguruskan hutang dengan lebih baik. Ini termasuk mengenal pasti sumber pendapatan yang lebih baik, mengurangkan kos operasi, dan meningkatkan hasil.

2. Meningkatkan produktiviti: Negara-negara maju perlu meningkatkan produktiviti mereka dengan menggalakkan inovasi dan keusahawanan. Ini akan membant

In [20]:
d = [
    {'role': 'user', 'content': 'code simple website hello world react, reply in jawi'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    repetition_penalty=1.2,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 21 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

code simple website hello world react, reply in jawi<|eot_id|><|start_header_id|>assistant<|end_header_id|>

اونتوق منچاڤاي لامن ويب هيللو ورلا دالم ريتيكل ايميج فيزيكال (REACT)⹁ اند بوليه مڠيکوتي بلوک بريکوت:

١. تمبهکن کومين "html" بارو جک اي تيدق وجود:
   ```jsx
   <div className="container">
     {/*... } */}
       <h1>Hello World</h1>
   </div>
   ```

٢. ݢوناکن JSX يڠ سواست اونتوق ممبوات ستروکتور HTML دان فوڠسي اوتوماتيق نۏيو-جسون ترسديا:

```jsx
import React from'react';

const HelloWorldPage = () => {
    return (
        <>
           <!--... }}
            <p>مرتي كموداه اين!</p>   
          </>
        />
      );
}

export default HelloWorldPage;
```

٣. توليس فايل `index.html` اتاو `index.js` سباليق-والاٴي ديريکتوري اوتمنتو اونتوق سويتوه اوجي.

٤. تتڤكن URL يڠ بتول دڠن رياکت:

دسيني⹂ اوبه سموا لاجو

In [29]:
d = [
    {'role': 'user', 'content': 'camne nk code react, reply in sabah'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    repetition_penalty=1.3,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 21 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk code react, reply in sabah<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Bah kunuk sia mau kasi tau macam mana mo bikin sama kod React dalam bahasaSabah. Tingu saja la:

1. Mula-muka dulu pakai package manager npm kalau ni belum ada lagi:
```bash
npm init -y # Bikin folder dengan laluan permintaan baru
```
2. Pasang semua perpustakaan yang kita perlukan (React Router, Axios untuk API):
```sh
cd my-app-react-project/
npx create-next app.js && cd./app 
npm install axios --save-dev // Pastikan sudah import dari pakej global ya sebelum jalankan Next App.
# Jangan buat pascafix kat terminal tu!
```

Jadi sini mungkin orang Sabah takut nak coding? Palui betul la kalau begitu... Kamurang faham ka apa yang saya cakap tadi?

Kalau masih nda faham, bilangkan ja! Nanti saya terangkan lain-lain cara.

**[I

In [30]:
d = [
    {'role': 'user', 'content': 'camne nk selesaikan masalah hutang negara, reply in mandarin'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 21 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk selesaikan masalah hutang negara, reply in mandarin<|eot_id|><|start_header_id|>assistant<|end_header_id|>

解决国家债务问题需要采取多方面的方法，包括提高透明度、促进诚信和问责制，以及实施有效的政策和战略。以下是一些建议：

1. 加强财政管理：政府应加强财政管理，确保公共部门的预算使用和支出是合理的。这可以通过建立健全的财务管理框架、监测和评估公共部门预算使用以及优先考虑可持续性和效率等措施来实现。
2. 提高透明度：政府应提高透明度，提供有关公共部门预算使用和支出的信息。这可以通过发布预算数据、报告财务活动以及举行公开听证会等方式来实现。
3. 执法和惩罚：政府应执法和惩罚腐败行为，特别是在公共部门。这可以通过设立独立调查机构、制定严厉的法律规定以及对腐败行为进行严厉处罚等措施来实现。
4. 增加收入：政府应增加收入，例如税收、公私合作伙伴关系（PPP）和其他收入来源，以减少公共部门的负担。这可以通过实施税收改革、鼓励创业和发展 PPP 等措施来实现。
5. 改善公共服务：政府应改善公共服务，例如教育、医疗保健和基础设施建设。这可以通过投资于基础设施建设、提高教育质量以及为公民提供更好的公共服务等措施来实现。

总之，解决国家债务问题需要采取全面和多方面的方法，包括提高透明度、促进诚信和问责制，以及实施有效的政策和战略。通过采取这些措施，政府可以帮助减轻国家债务负担并促进经济增长。<|eot_id|>


In [24]:
d = [
    {'role': 'user', 'content': 'camne nk selesaikan masalah hutang negara, reply in tamil'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=2048,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    repetition_penalty=1.1,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 21 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk selesaikan masalah hutang negara, reply in tamil<|eot_id|><|start_header_id|>assistant<|end_header_id|>

நீதியை நிவர்த்தி செய்வது என்பது கூட, இன்றைய நாடுகளில் உண்மையான அதிகாரங்களுடன் ஒழுங்கமைக்கப்பட்ட சட்டங்கள் மற்றும் அமலாக்க முறை பற்றிய போராட்டங்கள் உள்ளன. இந்த விஷயத்தில் பல விதமான நடவடிக்கைகள் உள்ளன:

1. **Debt Management Strategy**: முதன்மைக் குடும்ப முதலீட்டைப் பொறுத்தவரை, மாற்று முறையான முதலீட்டைப் பாதுகாப்பதாகக் கருதப்படும் கொள்கைகளை நீக்கவும் மறுசீரமைவுப்படுத்தவும் உள்ள நாடுகள் திறம்பட நடவடிக்கை எடுக்கின்றன. இந்த முதலீட்டைப் பாதுகாக்க அதன் முக்கியத்துவத்தை மேம்படுத்துவது, ஆபத்தான நிர்வாகத்தைப் பற்றி உறுதியாக வாதிடுவது மற்றும் குடும்பத்தினர்-மாணவர்கள் கையகப் பேச்சுக்கு ஏற்றவாறு முயற்சி செய்தல்.

2. **ஒத்துழைப்பு - ஓய்வு** மற்றும் மேம்பாடு (ODA) முறையை நியாயப்படுத்துதல்: ODA பல நாடுகள் பயன்படுத்தி 

In [35]:
d = [
    {'role': 'user', 'content': 'camne nk code simple hello world react, reply in perak'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=512,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    repetition_penalty=1.1,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 21 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk code simple hello world react, reply in perak<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Ate, teman nak buat skrip yang senang ni untuk kome. Tengok ye:

```javascript
console.log("Hello World dari React!");
```

Ni lah care mike boleh letak dalam HTML die:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Hello World dari React</title>
</head>
<body>
<script>
console.log("Hello World dari React!");
</script> <!-- Ni kod JavaScript je lah, tapi tak kene pakai JavaScript -->
</body>
</html>
```

Mike kene tambah ni kat paling bawah HTML tu: `<script>` ngan `<script>...</script>`. Kalo gune npm atau yarn utk nambahkan kebergantungan, pastikan dah ade tempat ujian.js.

Tapi ingat, walaupun buat macam ni senang je, kalau mike nak buat React yang lagi bagus, m

In [16]:
d = [
    {'role': 'user', 'content': 'camne nk code react'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    streamer=streamer
)
generation_output = ori_model.generate(**generate_kwargs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 21 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

camne nk code react<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Saya akan membantu anda dengan menghasilkan kod React yang mudah. Berikut adalah contoh kod untuk membuat komponen yang memaparkan senarai item:

```jsx
import React, { useState } from "react";

function ItemList() {
  const [items, setItems] = useState([
    { id: 1, name: 'Item 1' },
    { id: 2, name: 'Item 2' },
    { id: 3, name: 'Item 3' }
  ]);

  return (
    <div>
      {items.map((item) => (
        <p key={item.id}>{item.name}</p>
      ))}
    </div>
  );
}

export default ItemList;
```

Kod di atas menggunakan hook `useState` untuk menyimpan array item dalam keadaan dan menggunakannya sebagai prop ke komponen lain.

Untuk melaksanakan fungsi ini, anda perlu menambahkan komponen baru dalam projek React anda. Contohnya, jika an

In [17]:
ori_model = ori_model.to(torch.bfloat16)

In [18]:
ori_model.push_to_hub('mesolitica/malaysian-Llama-3.2-1B-Instruct')

README.md:   0%|          | 0.00/673 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-Llama-3.2-1B-Instruct/commit/2735bd550af3cb5dc31c58a37f1d1228ead60191', commit_message='Upload LlamaForCausalLM', commit_description='', oid='2735bd550af3cb5dc31c58a37f1d1228ead60191', pr_url=None, pr_revision=None, pr_num=None)